In [1]:
import kagglehub


In [2]:
import os
import json

# Replace with your actual Kaggle username and token
kaggle_username = "vuongtran11233"
kaggle_token = "9e534b1230ff13e77e96d939804a8c62"

# Define the path for kaggle.json
kaggle_path = os.path.join(os.path.expanduser("~"), ".kaggle")
kaggle_file = os.path.join(kaggle_path, "kaggle.json")

# Ensure the .kaggle directory exists
os.makedirs(kaggle_path, exist_ok=True)

# Write the credentials to kaggle.json
with open(kaggle_file, "w") as f:
    json.dump({"username": kaggle_username, "key": kaggle_token}, f)

# Set the appropriate permissions (for Linux/MacOS)
os.chmod(kaggle_file, 0o600)

print("Kaggle credentials saved successfully!")


Kaggle credentials saved successfully!


In [3]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

bkai_igh_neopolyp_path = kagglehub.competition_download('bkai-igh-neopolyp')

print('Data source import complete.')


100%|██████████| 314M/314M [00:03<00:00, 85.0MB/s]

Extracting files...


Data source import complete.


In [4]:
!pip install torchsummary
!pip install torchgeometry

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.7/42.7 kB 1.5 MB/s eta 0:00:00


In [5]:
!pip install segmentation-models-pytorch

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 23.1 MB/s eta 0:00:00
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16424 sha256=60b39ef95fe7dc63f46eca4ab485f76c86104193ed1770483b594b1c0338dca5
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60944 sha256=3c348b9596516df074a1eb192e34c6ef1f5e71b0bd0ba92974ebc1b81b271e49
  Stored in directory: /root/.cache/pip/wheels/35/cb/a5/8f534c60142835bfc889f9a482e4a67e0b817032d9c6883b64
Successfully built efficientnet-pytorch

In [6]:
!pip install torchinfo

In [7]:
import os
import pandas as pd
import numpy as np
import cv2
from torchvision.io import read_image
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, random_split, DataLoader
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

from torchvision.transforms import ToTensor
from PIL import Image
import os

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms
from torchinfo import summary
import timm
import segmentation_models_pytorch as smp
import wandb

/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [8]:
!nvidia-smi -L

/bin/bash: line 1: nvidia-smi: command not found


In [9]:
device = torch.device("cpu")
device

device(type='cpu')

In [10]:
class DatasetCustom(Dataset):
    def __init__(self, img_dir, label_dir, resize=None, transform=None):
        self.img_dir = img_dir
        self.label_dir = label_dir
        self.resize = resize
        self.transform = transform
        self.images = os.listdir(self.img_dir)

    def __len__(self):
        return len(self.images)

    def read_mask(self, mask_path):
        image = cv2.imread(mask_path)
        image = cv2.resize(image, self.resize)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

        lower_red1 = np.array([0, 100, 20])
        upper_red1 = np.array([10, 255, 255])
        lower_red2 = np.array([160,100,20])
        upper_red2 = np.array([179,255,255])

        lower_mask_red = cv2.inRange(image, lower_red1, upper_red1)
        upper_mask_red = cv2.inRange(image, lower_red2, upper_red2)

        red_mask = lower_mask_red + upper_mask_red
        red_mask[red_mask != 0] = 1

        green_mask = cv2.inRange(image, (36, 25, 25), (70, 255, 255))
        green_mask[green_mask != 0] = 2

        full_mask = cv2.bitwise_or(red_mask, green_mask)
        full_mask = np.expand_dims(full_mask, axis=-1)
        full_mask = full_mask.astype(np.uint8)

        return full_mask

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.images[idx])
        label_path = os.path.join(self.label_dir, self.images[idx])
        image = cv2.imread(img_path)  #  BGR
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # Convert to RGB
        label = self.read_mask(label_path)
        image = cv2.resize(image, self.resize)
        if self.transform:
            image = self.transform(image)

        return image, label

In [11]:
images_path = os.path.join(bkai_igh_neopolyp_path, "train/train")
image_path = []
TRAIN_DIR = os.path.join(bkai_igh_neopolyp_path, "train/train")
for root, dirs, files in os.walk(TRAIN_DIR):
    for file in files:
        path = os.path.join(root,file)
        image_path.append(path)

len(image_path)

1000

In [12]:
mask_path = []
TRAIN_MASK_DIR = os.path.join(bkai_igh_neopolyp_path, "train_gt/train_gt")
for root, dirs, files in os.walk(TRAIN_MASK_DIR):
    for file in files:
        path = os.path.join(root,file)
        mask_path.append(path)

len(mask_path)

1000

In [13]:
dataset = DatasetCustom(img_dir= TRAIN_DIR,
                             label_dir= TRAIN_MASK_DIR,
                             resize= (256,256),
                             transform = None)

In [14]:
batch_size = 32
images_data = []
labels_data = []
for x,y in dataset:
    images_data.append(x)
    labels_data.append(y)

In [15]:
import segmentation_models_pytorch as smp

model = smp.UnetPlusPlus(
    encoder_name="resnet18",
    encoder_weights="imagenet",
    in_channels=3,
    classes=3
)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 70.2MB/s]


In [16]:
class CustomDataset(Dataset):
    def __init__(self, data, targets, transform=None):
        self.data = data
        self.targets = targets
        self.transform = transform

    def __getitem__(self, index):
        image = self.data[index]
        label = self.targets[index]
        assert image.shape[:2] == label.shape[:2]
        if self.transform:
            transformed = self.transform(image=image, mask=label)
            image = transformed['image'].float()
            label = transformed['mask'].float()
            label = label.permute(2, 0, 1)
        return image, label

    def __len__(self):
        return len(self.data)

In [17]:
train_transformation = A.Compose([
    A.HorizontalFlip(p=0.4),
    A.VerticalFlip(p=0.4),
    A.RandomGamma (gamma_limit=(70, 130), eps=None, always_apply=False, p=0.2),
    A.RGBShift(p=0.3, r_shift_limit=10, g_shift_limit=10, b_shift_limit=10),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

val_transformation = A.Compose([
    A.Normalize(mean=(0.485, 0.456, 0.406),std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

<ipython-input-17-d765f530c16c>:4: UserWarning: Argument 'eps' is not valid and will be ignored.
  A.RandomGamma (gamma_limit=(70, 130), eps=None, always_apply=False, p=0.2),


In [18]:
train_size = int(0.8 * len(images_data))
val_size = len(images_data) - train_size
train_dataset = CustomDataset(images_data[:train_size], labels_data[:train_size], transform=train_transformation)
val_dataset = CustomDataset(images_data[train_size:], labels_data[train_size:], transform=val_transformation)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

In [19]:
learning_rate = 0.0001
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [20]:
color_dict= {0: (0, 0, 0),
             1: (255, 0, 0),
             2: (0, 255, 0)}
def mask_to_rgb(mask, color_dict):
    output = np.zeros((mask.shape[0], mask.shape[1], 3))

    for k in color_dict.keys():
        output[mask==k] = color_dict[k]

    return np.uint8(output)

In [ ]:
wandb.login(
    # set the wandb project where this run will be logged
#     project= "PolypSegment",
    key = "a752a545bfe8aec7b9d44039b5ca214bd32c26e5",
)
wandb.init(
    project = "polypsegment"
)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [ ]:
!pip install tqdm


In [ ]:
from tqdm import tqdm
import time

num_epochs = 200

device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
model.to(device)
criterion = nn.CrossEntropyLoss()
best_val_loss = 999

epoch_bar = tqdm(total=num_epochs, desc='Total Progress')

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        labels = labels.squeeze(dim=1).long()
        outputs = model(images)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        train_loss += loss.item()
    model.eval()
    with torch.no_grad():
        val_loss = 0
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)
            labels = labels.squeeze(dim=1).long()

            outputs = model(images)

            val_loss += criterion(outputs.float(),labels.long()).item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {val_loss/len(val_loader):.10f}")
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        checkpoint = {
            'epoch': epoch,
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'loss': val_loss,
        }
        save_path = f'model.pth'
        torch.save(checkpoint, save_path)

    epoch_bar.update(1)
    wandb.log({'Val_loss': val_loss/len(val_loader),'Train_loss': train_loss/len(train_loader)})
epoch_bar.close()


Total Progress:   0%|          | 0/200 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/wandb/sdk/wandb_run.py:2380: UserWarning: Run (4xzhv30h) is finished. The call to `_console_raw_callback` will be ignored. Please make sure that you are using an active run.
  lambda data: self._console_raw_callback("stderr", data),
Total Progress:   0%|          | 0/200 [02:04<?, ?it/s]


Epoch [1/200], Loss: 1.0748184834



Total Progress:   0%|          | 1/200 [00:24<1:21:22, 24.53s/it]

Epoch [2/200], Loss: 0.5467036196



Total Progress:   1%|          | 2/200 [00:46<1:15:35, 22.90s/it]

...

Epoch [152/200], Loss: 0.0744096536



Total Progress:  76%|███████▋  | 153/200 [57:53<17:42, 22.60s/it]

Epoch [153/200], Loss: 0.0733983149



Total Progress:  77%|███████▋  | 154/200 [58:16<17:19, 22.59s/it]

Epoch [154/200], Loss: 0.0725394700


In [39]:
# Explicitly map the model to CPU
checkpoint = torch.load('model.pth', map_location=torch.device('cpu'))
model.load_state_dict(checkpoint['model'])

# Set the device to CPU
device = torch.device('cpu')
model.to(device)


<ipython-input-39-f7f45449219b>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('model.pth', map_location=torch.device('cpu'))


UnetPlusPlus(
  (encoder): ResNetEncoder(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=Tru

In [ ]:
!mkdir prediction

In [41]:
model.eval()
test_path = os.path.join(bkai_igh_neopolyp_path, "test/test")
for i in os.listdir(test_path):
    img_path = os.path.join(test_path, i)
    ori_img = cv2.imread(img_path)
    ori_img = cv2.cvtColor(ori_img, cv2.COLOR_BGR2RGB)
    ori_w = ori_img.shape[0]
    ori_h = ori_img.shape[1]
    img = cv2.resize(ori_img, (256, 256))
    transformed = val_transformation(image=img)
    input_img = transformed["image"]
    input_img = input_img.unsqueeze(0).to(device)
    with torch.no_grad():
        output_mask = model.forward(input_img).squeeze(0).cpu().numpy().transpose(1,2,0)
    mask = cv2.resize(output_mask, (ori_h, ori_w))
    mask = np.argmax(mask, axis=2)
    mask_rgb = mask_to_rgb(mask, color_dict)
    mask_rgb = cv2.cvtColor(mask_rgb, cv2.COLOR_RGB2BGR)
    cv2.imwrite("prediction/{}".format(i), mask_rgb)

In [44]:
def rle_to_string(runs):
    return ' '.join(str(x) for x in runs)

def rle_encode_one_mask(mask):
    pixels = mask.flatten()
    pixels[pixels > 225] = 255
    pixels[pixels <= 225] = 0
    use_padding = False
    if pixels[0] or pixels[-1]:
        use_padding = True
        pixel_padded = np.zeros([len(pixels) + 2], dtype=pixels.dtype)
        pixel_padded[1:-1] = pixels
        pixels = pixel_padded
    rle = np.where(pixels[1:] != pixels[:-1])[0] + 2
    if use_padding:
        rle = rle - 1
    rle[1::2] = rle[1::2] - rle[:-1:2]

    return rle_to_string(rle)

def rle2mask(mask_rle, shape=(3,3)):
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T

def mask2string(dir):
    strings = []
    ids = []
    ws, hs = [[] for i in range(2)]
    for image_id in os.listdir(dir):
        id = image_id.split('.')[0]
        path = os.path.join(dir, image_id)
        print(path)
        img = cv2.imread(path)[:,:,::-1]
        h, w = img.shape[0], img.shape[1]
        for channel in range(2):
            ws.append(w)
            hs.append(h)
            ids.append(f'{id}_{channel}')
            string = rle_encode_one_mask(img[:,:,channel])
            strings.append(string)
    r = {
        'ids': ids,
        'strings': strings,
    }
    return r


MASK_DIR_PATH = '/content/prediction'
dir = MASK_DIR_PATH
res = mask2string(dir)
df = pd.DataFrame(columns=['Id', 'Expected'])
df['Id'] = res['ids']
df['Expected'] = res['strings']

df.to_csv(r'output.csv', index=False)

/content/prediction/d6bf62f215f0da4ad3a7ab8df9da7386.jpeg
/content/prediction/cf6644589e532a9ee954f81faedbce39.jpeg
/content/prediction/2ed9fbb63b28163a745959c03983064a.jpeg
/content/prediction/3b8318ecf467d7ad048df39beb176363.jpeg
/content/prediction/4baddc22268d4b4ef4d95ceea1195799.jpeg
/content/prediction/cdf3f33c3ca4d5060a633a8d5b2b2b55.jpeg
/content/prediction/780fd497e1c0e9082ea2c193ac8d551c.jpeg
/content/prediction/f7fdb2d45b21960c94b0aab4c024a573.jpeg
/content/prediction/7330398846f67b5df7cdf3f33c3ca4d5.jpeg
/content/prediction/eecd70ebce6347c491b37c8c2e5a64a8.jpeg
/content/prediction/85a04faeeb2b535797395305af926a6f.jpeg
/content/prediction/c656702fa602bb3c7abacdbd7e6afd56.jpeg
/content/prediction/936de314f2d95e6c487ffa651b477422.jpeg
/content/prediction/f13dd311a65d2b46d0a6085835c525af.jpeg
/content/prediction/5664c1711b62f15ec83b97bb11e8e0c4.jpeg
/content/prediction/05b78a91391adc0bb223c4eaf3372eae.jpeg
/content/prediction/e1797c77826f9a7021bab9fc73303988.jpeg
/content/predi

#Infer

In [40]:
import argparse
import os
import torch
import cv2
from random import choice
import numpy as np
import segmentation_models_pytorch as smp
from collections import OrderedDict
from torchvision import transforms


def preprocess_image(image_path, input_size=(256, 256)):
    """
    Preprocess the input image for the model.
    Resize, normalize, and convert to a tensor.
    """
    # Read the image
    image = cv2.imread(image_path)
    if image is None:
        raise FileNotFoundError(f"Image not found at path: {image_path}")
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Resize image
    resized = cv2.resize(image, input_size, interpolation=cv2.INTER_AREA)

    # Normalize and convert to tensor
    preprocess = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    tensor = preprocess(resized).unsqueeze(0)  # Add batch dimension
    return tensor, image.shape[:2]  # Return tensor and original shape


def postprocess_output(output, original_shape):
    """
    Post-process the model's output to match the original image shape.
    Converts logits to binary mask and resizes back.
    """
    # Convert logits to probabilities and threshold at 0.5
    output = torch.sigmoid(output).squeeze(0).detach().cpu().numpy()
    mask = (output > 0.5).astype(np.uint8)

    # Resize mask to original shape
    mask_resized = cv2.resize(mask.transpose(1, 2, 0), original_shape[::-1], interpolation=cv2.INTER_NEAREST)

    return mask_resized


def main():
    # Parse arguments
    parser = argparse.ArgumentParser(description="Run segmentation inference on a single image.")
    parser.add_argument("--image_path", type=str, required=True, help="Path to the input image.")
    parser.add_argument("--output_path", type=str, default="segmented_output.png", help="Path to save the output image.")
    args = parser.parse_args()

    # Load model
    model = smp.UnetPlusPlus(
        encoder_name="resnet18",
        encoder_weights=None,  # We're loading weights from the checkpoint
        in_channels=3,
        classes=3  # Number of output segmentation classes
    )

    # Load checkpoint
    # Explicitly map the model to CPU
    checkpoint = torch.load('model.pth', map_location=torch.device('cpu'))
    model.load_state_dict(checkpoint['model'])

    # Set the device to CPU
    device = torch.device('cpu')
    model.to(device)
    model.eval()  # Set model to evaluation mode

    # Preprocess input image
    image_tensor, original_shape = preprocess_image(args.image_path)

    # Perform inference
    with torch.no_grad():
        output = model(image_tensor)

    # Postprocess output
    segmented_image = postprocess_output(output, original_shape)

    # Save the segmented image
    segmented_image_colored = (segmented_image * 255).astype(np.uint8)  # Scale to 0-255
    cv2.imwrite(args.output_path, segmented_image_colored)

    print(f"Segmented image saved to: {args.output_path}")



<ipython-input-40-3a8471058fde>:67: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('model.pth', map_location=torch.device('cpu'))


Segmented image saved to: /content/output.png


### Code snippet for run in a notebook

In [ ]:
import sys

test_path = os.path.join(bkai_igh_neopolyp_path, "test/test")
i = choice(os.listdir(test_path))
img_path = os.path.join(test_path, i)
cv2.imwrite("input.png", cv2.imread(img_path))
# Mock command-line arguments
sys.argv = [
    "infer.py",
    "--image_path", img_path,
    "--output_path", "/content/output.png",
]

# Call the main function
if __name__ == "__main__":
    main()
